In [1]:
import glob
import pandas as pd
from pathlib import Path

In [2]:
datasets= glob.glob("./jailbreak-datasets/*/results.csv")

In [3]:
dfs = [pd.read_csv(dataset) for dataset in datasets]

In [8]:
all_fails = []
for df in dfs:
    failure_prompts = df[df['Success'] == 'Failure']['Jailbreak Prompt'].head(50)
    failure_prompts = failure_prompts.to_list()
    all_fails.extend(failure_prompts)

In [9]:
df_fails = pd.DataFrame(all_fails, columns=["prompt"])
df_fails['label'] = True

In [10]:
df_safe = pd.read_csv("./data/benign_prompts_supplemental.csv")
df_safe = df_safe.head(len(df_fails))
df_safe['label'] = False

In [16]:
len(df_safe), len(df_fails), len(df_safe) + len(df_fails)

(270, 270, 540)

In [13]:
from sklearn.model_selection import train_test_split
df_90_fails, df_10_fails = train_test_split(df_fails, test_size=0.1, random_state=42)
df_90_safe, df_10_safe = train_test_split(df_safe, test_size=0.1, random_state=42)

In [14]:
df_train = pd.read_csv("./data/full_v1/train.csv")
df_test = pd.read_csv("./data/full_v1/test.csv")
df_combined_train = pd.concat([df_90_fails, df_90_safe, df_train], ignore_index=True)
df_combined_train["label"] = df_combined_train["label"].astype(bool)
df_combined_test = pd.concat([df_10_fails, df_10_safe, df_test], ignore_index=True)
df_combined_test["label"] = df_combined_test["label"].astype(bool)

In [19]:
df_combined_train.to_csv("./data/full_v4/train.csv")

In [20]:
df_combined_test.to_csv("./data/full_v4/test.csv")